# FIT3182 Assignment 3 Part A

- Name: Ong Di Sheng
- Student ID: 31109667
- Email: dong0009@student.monash.edu

## Task 1: MongoDB Data Model 

### ***1. Based on the two data sets provided i.e. hotspot_historic.csv and climate_historic.csv, design a suitable data model to support the efficient querying of the two data sets in MongoDB. Justify your data model design.*** ###

### Answer ###

Before answering this question, let's first examine the data from both csv files to gain a better understanding of its structure, content and relationships by using pandas library.

In [1]:
# import libraries 
import pandas as pd
import pymongo

# show first 5 records from historic climate dataset
climate = pd.read_csv('climate_historic.csv')
climate.head()

,station,date,air_temperature_celcius,relative_humidity,windspeed_knots,max_wind_speed,precipitation,GHI_w/m2
0,948700,31/12/2021,19,56.8,7.9,11.1,0.00I,154
1,948700,2/1/2022,15,50.7,9.2,13.0,0.02G,128
2,948700,3/1/2022,16,53.6,8.1,15.0,0.00G,133
3,948700,4/1/2022,24,61.6,7.7,14.0,0.00I,186
4,948700,5/1/2022,24,62.3,7.0,13.0,0.00I,185


In [2]:
# show first 5 records from historic hotspot dataset
hotspot = pd.read_csv('hotspot_historic.csv')
hotspot.head()

,latitude,longitude,datetime,confidence,date,surface_temperature_celcius
0,-37.966,145.051,2022-12-27T04:16:51,78,27/12/2022,68
1,-35.541,143.311,2022-12-27T00:02:15,82,27/12/2022,63
2,-35.554,143.307,2022-12-27T00:02:15,67,27/12/2022,53
3,-35.543,143.316,2022-12-27T00:02:14,86,27/12/2022,67
4,-37.708,145.100,2022-12-25T04:29:08,80,25/12/2022,54


Based on the assignment specifications, there exists a relationship between climate and hotspot data. A climate data entry can have zero or multiple corresponding hotspot data entries, depending on the occurrence of fires on a specific day. To establish this connection, we can utilize the date field as a shared identifier. In the provided example, I have specifically selected rows from both datasets that have the date 29/3/2022 as a reference point for building the data model.

In [3]:
climate[climate['date'] == '29/3/2022']

,station,date,air_temperature_celcius,relative_humidity,windspeed_knots,max_wind_speed,precipitation,GHI_w/m2
87,948701,29/3/2022,17,49.9,12.2,21.0,0.24G,146


In [4]:
hotspot[hotspot['date'] == '29/3/2022']

,latitude,longitude,datetime,confidence,date,surface_temperature_celcius
2511,-34.2648,141.6325,2022-03-29T00:48:40,69,29/3/2022,51


Based on the provided sample data, we can create an example representation of how the collections might be structured in MongoDB. 

#### Climate Collection: ####
```
{
   'GHI_w/m2': 146,
   '_id': ObjectId('64794430094ec77d3a57cc2e'),
   'air_temperature_celcius': 17,
   'date': datetime.datetime(2022, 3, 29, 0, 0),
   'max_wind_speed': 21.0,
   'precipitation': 0.24,
   'precipitation_flag': 'G',
   'relative_humidity': 49.9,
   'station': 948701,
   'windspeed_knots': 12.2
}
```

#### Hotspot Collection: ####
```
{
   'confidence': 69,
   'date': datetime.datetime(2022, 3, 29, 0, 0),
   'datetime': datetime.datetime(2022, 3, 29, 0, 48, 40),
   'latitude': -34.2648,
   'longitude': 141.6325,
   'surface_temperature_celcius': 51
}
```

After careful consideration, I have decided to construct an embedded data model for this assignment in order to support the efficient querying of the two datasets in MongoDB as shown below.

#### `Example of Embedded Data Model:` ####
```
{
   'GHI_w/m2': 146,
   '_id': ObjectId('64794430094ec77d3a57cc2e'),
   'air_temperature_celcius': 17,
   'date': datetime.datetime(2022, 3, 29, 0, 0),
   'max_wind_speed': 21.0,
   'precipitation': 0.24,
   'precipitation_flag': 'G',
   'relative_humidity': 49.9,
   'station': 948701,
   'windspeed_knots': 12.2,
   'hotspots': [
       {
           'confidence': 69,
           'date': datetime.datetime(2022, 3, 29, 0, 0),
           'datetime': datetime.datetime(2022, 3, 29, 0, 48, 40),
           'latitude': -34.2648,
           'longitude': 141.6325,
           'surface_temperature_celcius': 51
       }
   ]
}
```

#### `Justification:` ####

Based on initial observations of the historic climate and hotspot data, it is apparent that there is consistently only one climate entry per day. However, there can be multiple hotspot entries recorded for a single day. This suggests a one-to-many relationship from climate data to hotspot data. Considering this relationship, an embedded data model is more suitable. In this proposed embedded data model, each climate data entry will include an array called "hotspots". For every hotspot data entry with the same date as the climate data, it will be appended to this array. 

One of the advantages of using an embedded data model is the convenience it offers by having all the required data in a single collection. In the case of Task 2 below, where querying data from both csv files is necessary, opting for a referencing data model would involve performing several left outer join operations using aggregations. As the dataset grows larger, this approach can become expensive and time-consuming due to the total join cost. On the other hand, with an embedded data model, there is no need to perform any join operations since all the relevant information is stored within one collection. This approach significantly reduces computational complexity and simplifies the querying workflow by eliminating the need for separate lookups or complex join operations.

While an embedded data model offers numerous advantages, there are certain drawbacks to consider. One notable downside is that the size of the document can increase significantly if there are numerous hotspot entries associated with a single climate entry. However, in our specific case, this is not a major concern since MongoDB's document size limit of 16MB is more than sufficient to accommodate a substantial amount of hotspot data as it is highly unlikely to encounter a scenario where thousands of fires break out across Victoria in a single day.

Based on the aforementioned considerations, I am confident that the embedded data model is the most suitable choice for storing climate and hotspot data in our specific application. Therefore, I have made the decision to utilize this model for our project.

## Task 2: Querying MongoDB using PyMongo 

### ***1. Write a python program that will read the data from hotspot_historic.csv and climate_historic.csv and load them to the new database (e.g. fit3182_assignment_db). The collection(s) in fit3182_assignment_db will be based on the document model you have designed in Task A1.*** ###

In [5]:
# import libraries 
import pandas as pd
from pymongo import MongoClient
from pprint import pprint
from datetime import datetime

# read data from csv files
climate = pd.read_csv('climate_historic.csv')
hotspot = pd.read_csv('hotspot_historic.csv')

# identify data types
# print(climate.dtypes)
# print(hotspot.dtypes)

# convert data types (datetime)
climate['date'] = pd.to_datetime(climate['date'], format='%d/%m/%Y')
hotspot['date'] = pd.to_datetime(hotspot['date'], format='%d/%m/%Y')
hotspot['datetime'] = pd.to_datetime(hotspot['datetime'], format='%Y-%m-%dT%H:%M:%S')

# format date columns
climate['date'] = climate['date'].dt.strftime('%d-%m-%Y')
hotspot['date'] = hotspot['date'].dt.strftime('%d-%m-%Y')
hotspot['datetime'] = hotspot['datetime'].dt.strftime('%Y-%m-%dT%H:%M:%S')

# construct embedded model 
# iterate over every record from climate data (as a dictionary)
climate_dict = climate.to_dict('records')
for climate_doc in climate_dict:
        
    # remove whitespace in `precipitation `
    climate_doc['precipitation'] = climate_doc['precipitation ']
    del climate_doc['precipitation ']
    
    # separate `precipitation` into value and flag
    precipitation = climate_doc['precipitation']
    climate_doc['precipitation'] = float(precipitation[:-1])
    climate_doc['precipitation_flag'] = precipitation[-1]
    
    # retrieve hotspot record(s) with matching date as current climate
    matching_hotspot = hotspot[hotspot['date'] == climate_doc['date']]
    
    # update climate date format before inserting into db
    climate_doc['date'] = datetime.strptime(climate_doc['date'], '%d-%m-%Y')
    
    # to be embedded with hotspot documents
    climate_doc['hotspots'] = []
    
    # iterate over every record from matching hotspot data (as a dictionary)
    for hotspot_doc in matching_hotspot.to_dict('records'):
        
        # update hotspot date format before inserting into db
        hotspot_doc['date'] = datetime.strptime(hotspot_doc['date'], '%d-%m-%Y')
        hotspot_doc['datetime'] = datetime.strptime(hotspot_doc['datetime'], '%Y-%m-%dT%H:%M:%S')
        
        # embed with hotspot document
        climate_doc['hotspots'].append(hotspot_doc)
    

In [6]:
# create connection with mongodb
# replace with your own IP address and port number
client = MongoClient('192.168.1.110', 27017)

# access database 
db = client.fit3182_assignment_db

# access collection
collection = db.embedded_climate_hotspot

# drop collection if it already exists in db
collection.drop()

# load processed data into db collection
result = collection.insert_many(climate_dict)

# show all newly inserted documents
for document in collection.find():
    pprint(document)
    

{'GHI_w/m2': 154,
 '_id': ObjectId('6481357e25c02be8d1a097ad'),
 'air_temperature_celcius': 19,
 'date': datetime.datetime(2021, 12, 31, 0, 0),
 'hotspots': [],
 'max_wind_speed': 11.1,
 'precipitation': 0.0,
 'precipitation_flag': 'I',
 'relative_humidity': 56.8,
 'station': 948700,
 'windspeed_knots': 7.9}
{'GHI_w/m2': 128,
 '_id': ObjectId('6481357e25c02be8d1a097ae'),
 'air_temperature_celcius': 15,
 'date': datetime.datetime(2022, 1, 2, 0, 0),
 'hotspots': [],
 'max_wind_speed': 13.0,
 'precipitation': 0.02,
 'precipitation_flag': 'G',
 'relative_humidity': 50.7,
 'station': 948700,
 'windspeed_knots': 9.2}
{'GHI_w/m2': 133,
 '_id': ObjectId('6481357e25c02be8d1a097af'),
 'air_temperature_celcius': 16,
 'date': datetime.datetime(2022, 1, 3, 0, 0),
 'hotspots': [],
 'max_wind_speed': 15.0,
 'precipitation': 0.0,
 'precipitation_flag': 'G',
 'relative_humidity': 53.6,
 'station': 948700,
 'windspeed_knots': 8.1}
{'GHI_w/m2': 186,
 '_id': ObjectId('6481357e25c02be8d1a097b0'),
 'air_tem

               'date': datetime.datetime(2022, 5, 3, 0, 0),
               'datetime': datetime.datetime(2022, 5, 3, 4, 2, 20),
               'latitude': -36.3415,
               'longitude': 141.5733,
               'surface_temperature_celcius': 64},
              {'confidence': 65,
               'date': datetime.datetime(2022, 5, 3, 0, 0),
               'datetime': datetime.datetime(2022, 5, 3, 4, 2, 20),
               'latitude': -36.3884,
               'longitude': 141.0322,
               'surface_temperature_celcius': 42},
              {'confidence': 73,
               'date': datetime.datetime(2022, 5, 3, 0, 0),
               'datetime': datetime.datetime(2022, 5, 3, 4, 2, 20),
               'latitude': -36.0106,
               'longitude': 144.9927,
               'surface_temperature_celcius': 47},
              {'confidence': 84,
               'date': datetime.datetime(2022, 5, 3, 0, 0),
               'datetime': datetime.datetime(2022, 5, 3, 4, 2, 20),
           

### ***2. Write queries to answer the following tasks on fit3182_assignment_db and corresponding collection(s). You need to write the queries as a python program using the pymongo library in Jupyter Notebook.*** ###

*a.	Find climate data on 12th December 2022.*

In [7]:
# convert target date to datetime object
target_date = datetime.strptime('12-12-2022', '%d-%m-%Y')

# retrieve documents with matching target date
# hotspot data is not needed
result = collection.find({'date': target_date}, {'_id': 0, 'hotspots': 0})

for document in result:
    # format date to be more readable 
    document['date'] = document['date'].strftime('%d-%m-%Y')
    pprint(document)

{'GHI_w/m2': 156,
 'air_temperature_celcius': 19,
 'date': '12-12-2022',
 'max_wind_speed': 12.0,
 'precipitation': 0.0,
 'precipitation_flag': 'I',
 'relative_humidity': 55.3,
 'station': 948702,
 'windspeed_knots': 6.2}


*b. Find the latitude, longitude, surface temperature (°C), and confidence when the surface temperature (°C) was between 65 °C and 100 °C.*

In [8]:
result = collection.aggregate([
    # flatten associative array of hotspots 
    {'$unwind': '$hotspots'},
    # filter by matching surface temperature 
    {'$match': {'hotspots.surface_temperature_celcius': {'$gte': 65, '$lte': 100}}},
    # output necessary fields only
    {'$project': {
            '_id': 0, 
            'hotspots.latitude': 1, 
            'hotspots.longitude': 1, 
            'hotspots.surface_temperature_celcius': 1, 
            'hotspots.confidence': 1
        }
    }
])

for document in result:
    pprint(document)

{'hotspots': {'confidence': 94,
              'latitude': -37.2284,
              'longitude': 147.9187,
              'surface_temperature_celcius': 73}}
{'hotspots': {'confidence': 97,
              'latitude': -37.6572,
              'longitude': 142.0703,
              'surface_temperature_celcius': 80}}
{'hotspots': {'confidence': 84,
              'latitude': -37.0193,
              'longitude': 148.1459,
              'surface_temperature_celcius': 71}}
{'hotspots': {'confidence': 100,
              'latitude': -37.4229,
              'longitude': 147.027,
              'surface_temperature_celcius': 99}}
{'hotspots': {'confidence': 80,
              'latitude': -37.0055,
              'longitude': 148.1582,
              'surface_temperature_celcius': 68}}
{'hotspots': {'confidence': 85,
              'latitude': -37.4128,
              'longitude': 147.0242,
              'surface_temperature_celcius': 98}}
{'hotspots': {'confidence': 90,
              'latitude': -34.357,
   

*c. Find date, surface temperature (°C), air temperature (°C), relative humidity and max wind speed on 15th and 16th of December 2022.*

In [9]:
# convert target dates to datetime objects
target_date1 = datetime.strptime('15-12-2022', '%d-%m-%Y')
target_date2 = datetime.strptime('16-12-2022', '%d-%m-%Y')

result = collection.find(
    {'$or': [{'date': target_date1}, {'date': target_date2}]},
    # output necessary fields only
    {'_id': 0, 'date': 1, 'hotspots.surface_temperature_celcius': 1, 'air_temperature_celcius': 1,
     'relative_humidity': 1, 'max_wind_speed': 1}
)


for document in result:
    pprint(document)

{'air_temperature_celcius': 18,
 'date': datetime.datetime(2022, 12, 15, 0, 0),
 'hotspots': [{'surface_temperature_celcius': 42},
              {'surface_temperature_celcius': 36},
              {'surface_temperature_celcius': 38},
              {'surface_temperature_celcius': 40}],
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0}
{'air_temperature_celcius': 18,
 'date': datetime.datetime(2022, 12, 16, 0, 0),
 'hotspots': [{'surface_temperature_celcius': 43},
              {'surface_temperature_celcius': 33},
              {'surface_temperature_celcius': 54},
              {'surface_temperature_celcius': 73},
              {'surface_temperature_celcius': 55},
              {'surface_temperature_celcius': 75},
              {'surface_temperature_celcius': 55},
              {'surface_temperature_celcius': 66},
              {'surface_temperature_celcius': 56},
              {'surface_temperature_celcius': 60},
              {'surface_temperature_celcius': 73},
              {'surfa

*d. Find datetime, air temperature (°C), surface temperature (°C) and confidence when the confidence is between 80 and 100.*

In [10]:
result = collection.aggregate([
    # flatten associative array of hotspots 
    {'$unwind': '$hotspots'},
    # filter by matching confidence  
    {'$match': {'hotspots.confidence': {'$gte': 80, '$lte': 100}}},
    # output necessary fields only
    {'$project': {
            '_id': 0, 
            'hotspots.datetime': 1, 
            'air_temperature_celcius': 1, 
            'hotspots.surface_temperature_celcius': 1, 
            'hotspots.confidence': 1
        }
    }
])

for document in result:
    pprint(document)

{'air_temperature_celcius': 20,
 'hotspots': {'confidence': 87,
              'datetime': datetime.datetime(2022, 3, 6, 5, 6, 30),
              'surface_temperature_celcius': 62}}
{'air_temperature_celcius': 20,
 'hotspots': {'confidence': 85,
              'datetime': datetime.datetime(2022, 3, 6, 5, 6, 20),
              'surface_temperature_celcius': 59}}
{'air_temperature_celcius': 19,
 'hotspots': {'confidence': 88,
              'datetime': datetime.datetime(2022, 3, 7, 4, 16, 10),
              'surface_temperature_celcius': 64}}
{'air_temperature_celcius': 23,
 'hotspots': {'confidence': 86,
              'datetime': datetime.datetime(2022, 3, 9, 13, 23, 40),
              'surface_temperature_celcius': 41}}
{'air_temperature_celcius': 19,
 'hotspots': {'confidence': 100,
              'datetime': datetime.datetime(2022, 3, 10, 4, 48, 40),
              'surface_temperature_celcius': 105}}
{'air_temperature_celcius': 19,
 'hotspots': {'confidence': 100,
              'datetime

              'surface_temperature_celcius': 58}}
{'air_temperature_celcius': 19,
 'hotspots': {'confidence': 100,
              'datetime': datetime.datetime(2022, 4, 6, 4, 20, 50),
              'surface_temperature_celcius': 95}}
{'air_temperature_celcius': 19,
 'hotspots': {'confidence': 88,
              'datetime': datetime.datetime(2022, 4, 6, 4, 20, 50),
              'surface_temperature_celcius': 65}}
{'air_temperature_celcius': 19,
 'hotspots': {'confidence': 83,
              'datetime': datetime.datetime(2022, 4, 6, 4, 20, 50),
              'surface_temperature_celcius': 63}}
{'air_temperature_celcius': 19,
 'hotspots': {'confidence': 81,
              'datetime': datetime.datetime(2022, 4, 6, 4, 20, 40),
              'surface_temperature_celcius': 60}}
{'air_temperature_celcius': 19,
 'hotspots': {'confidence': 83,
              'datetime': datetime.datetime(2022, 4, 6, 4, 20, 40),
              'surface_temperature_celcius': 78}}
{'air_temperature_celcius': 19,
 'hotsp

*e.	Find the top 10 records with the highest surface temperature (°C).*

In [11]:
result = collection.aggregate([
    # flatten associative array of hotspots 
    {'$unwind': '$hotspots'},
    # sort by surface temperature in decreasing order
    {'$sort': {'hotspots.surface_temperature_celcius': -1}},
    # retrieve top 10 records only
    {'$limit': 10}
])


for document in result:
    pprint(document)

{'GHI_w/m2': 122,
 '_id': ObjectId('6481357e25c02be8d1a09818'),
 'air_temperature_celcius': 15,
 'date': datetime.datetime(2022, 4, 18, 0, 0),
 'hotspots': {'confidence': 100,
              'date': datetime.datetime(2022, 4, 18, 0, 0),
              'datetime': datetime.datetime(2022, 4, 18, 4, 52),
              'latitude': -38.1665,
              'longitude': 143.062,
              'surface_temperature_celcius': 124},
 'max_wind_speed': 9.9,
 'precipitation': 0.0,
 'precipitation_flag': 'I',
 'relative_humidity': 56.1,
 'station': 948701,
 'windspeed_knots': 5.1}
{'GHI_w/m2': 140,
 '_id': ObjectId('6481357e25c02be8d1a0980a'),
 'air_temperature_celcius': 16,
 'date': datetime.datetime(2022, 4, 4, 0, 0),
 'hotspots': {'confidence': 100,
              'date': datetime.datetime(2022, 4, 4, 0, 0),
              'datetime': datetime.datetime(2022, 4, 4, 4, 32, 50),
              'latitude': -36.343,
              'longitude': 142.1986,
              'surface_temperature_celcius': 123},
 'm

*f. Find the number of fires each day. You are required to only display the total number of fires and the date in the output.*

In [12]:
result = collection.aggregate([
    {'$project': {'_id': 0, 'date': 1, 'number_of_fires': {'$size': '$hotspots'}}}
])

for document in result:
    # format date to be more readable 
    document['date'] = document['date'].strftime('%d-%m-%Y')
    pprint(document)

{'date': '31-12-2021', 'number_of_fires': 0}
{'date': '02-01-2022', 'number_of_fires': 0}
{'date': '03-01-2022', 'number_of_fires': 0}
{'date': '04-01-2022', 'number_of_fires': 0}
{'date': '05-01-2022', 'number_of_fires': 0}
{'date': '06-01-2022', 'number_of_fires': 0}
{'date': '07-01-2022', 'number_of_fires': 0}
{'date': '08-01-2022', 'number_of_fires': 0}
{'date': '09-01-2022', 'number_of_fires': 0}
{'date': '10-01-2022', 'number_of_fires': 0}
{'date': '11-01-2022', 'number_of_fires': 0}
{'date': '12-01-2022', 'number_of_fires': 0}
{'date': '13-01-2022', 'number_of_fires': 0}
{'date': '14-01-2022', 'number_of_fires': 0}
{'date': '15-01-2022', 'number_of_fires': 0}
{'date': '16-01-2022', 'number_of_fires': 0}
{'date': '17-01-2022', 'number_of_fires': 0}
{'date': '18-01-2022', 'number_of_fires': 0}
{'date': '19-01-2022', 'number_of_fires': 0}
{'date': '20-01-2022', 'number_of_fires': 0}
{'date': '21-01-2022', 'number_of_fires': 0}
{'date': '22-01-2022', 'number_of_fires': 0}
{'date': '

*g. Find the records of fires where the confidence is below 70.*

In [13]:
result = collection.aggregate([
    # flatten associative array of hotspots 
    {'$unwind': '$hotspots'},
    # filter by matching confidence 
    {'$match': {'hotspots.confidence': {'$lt': 70}}},
    # output fire records only
    {'$project': {'_id': 0, 'hotspots': 1}}
])

for document in result:
    pprint(document)

{'hotspots': {'confidence': 68,
              'date': datetime.datetime(2022, 3, 8, 0, 0),
              'datetime': datetime.datetime(2022, 3, 8, 4, 51),
              'latitude': -37.7885,
              'longitude': 141.9352,
              'surface_temperature_celcius': 55}}
{'hotspots': {'confidence': 54,
              'date': datetime.datetime(2022, 3, 9, 0, 0),
              'datetime': datetime.datetime(2022, 3, 9, 3, 57),
              'latitude': -37.7171,
              'longitude': 147.5866,
              'surface_temperature_celcius': 44}}
{'hotspots': {'confidence': 55,
              'date': datetime.datetime(2022, 3, 10, 0, 0),
              'datetime': datetime.datetime(2022, 3, 10, 4, 43),
              'latitude': -36.2544,
              'longitude': 148.0353,
              'surface_temperature_celcius': 42}}
{'hotspots': {'confidence': 54,
              'date': datetime.datetime(2022, 3, 10, 0, 0),
              'datetime': datetime.datetime(2022, 3, 10, 4, 42, 30),
   

{'hotspots': {'confidence': 57,
              'date': datetime.datetime(2022, 4, 12, 0, 0),
              'datetime': datetime.datetime(2022, 4, 12, 13, 9, 20),
              'latitude': -35.4407,
              'longitude': 142.3644,
              'surface_temperature_celcius': 35}}
{'hotspots': {'confidence': 67,
              'date': datetime.datetime(2022, 4, 12, 0, 0),
              'datetime': datetime.datetime(2022, 4, 12, 5, 28),
              'latitude': -37.4755,
              'longitude': 142.6339,
              'surface_temperature_celcius': 43}}
{'hotspots': {'confidence': 54,
              'date': datetime.datetime(2022, 4, 12, 0, 0),
              'datetime': datetime.datetime(2022, 4, 12, 5, 26, 50),
              'latitude': -36.7453,
              'longitude': 141.4578,
              'surface_temperature_celcius': 39}}
{'hotspots': {'confidence': 50,
              'date': datetime.datetime(2022, 4, 12, 0, 0),
              'datetime': datetime.datetime(2022, 4, 12, 5, 

              'date': datetime.datetime(2022, 4, 18, 0, 0),
              'datetime': datetime.datetime(2022, 4, 18, 4, 44, 50),
              'latitude': -37.441,
              'longitude': 142.9966,
              'surface_temperature_celcius': 42}}
{'hotspots': {'confidence': 69,
              'date': datetime.datetime(2022, 4, 18, 0, 0),
              'datetime': datetime.datetime(2022, 4, 18, 4, 44, 50),
              'latitude': -37.4007,
              'longitude': 142.7674,
              'surface_temperature_celcius': 44}}
{'hotspots': {'confidence': 54,
              'date': datetime.datetime(2022, 4, 18, 0, 0),
              'datetime': datetime.datetime(2022, 4, 18, 4, 44, 50),
              'latitude': -37.4984,
              'longitude': 142.0686,
              'surface_temperature_celcius': 40}}
{'hotspots': {'confidence': 68,
              'date': datetime.datetime(2022, 4, 18, 0, 0),
              'datetime': datetime.datetime(2022, 4, 18, 4, 44, 50),
              'latit

*h. Find the average surface temperature (°C) for each day. You are required to only display average surface temperature (°C) and the date in the output.*

In [14]:
result = collection.aggregate([
    # ensure climate data with empty hotspot will be included after unwind
    {'$unwind': {
        'path': '$hotspots',
        'preserveNullAndEmptyArrays': True
    }},
    # compute average surface temperature for each day
    {'$group': {'_id': '$date', 'avg': {'$avg': '$hotspots.surface_temperature_celcius'}}},
    # rename fields
    {'$project': {'date': '$_id', 'avg_surface_temperature': '$avg', '_id': 0}},
    # sort by increasing date 
    {'$sort': {'date': 1}}
])

for document in result:
    # format date to be more readable 
    document['date'] = document['date'].strftime('%d-%m-%Y')
    pprint(document)

{'avg_surface_temperature': None, 'date': '31-12-2021'}
{'avg_surface_temperature': None, 'date': '02-01-2022'}
{'avg_surface_temperature': None, 'date': '03-01-2022'}
{'avg_surface_temperature': None, 'date': '04-01-2022'}
{'avg_surface_temperature': None, 'date': '05-01-2022'}
{'avg_surface_temperature': None, 'date': '06-01-2022'}
{'avg_surface_temperature': None, 'date': '07-01-2022'}
{'avg_surface_temperature': None, 'date': '08-01-2022'}
{'avg_surface_temperature': None, 'date': '09-01-2022'}
{'avg_surface_temperature': None, 'date': '10-01-2022'}
{'avg_surface_temperature': None, 'date': '11-01-2022'}
{'avg_surface_temperature': None, 'date': '12-01-2022'}
{'avg_surface_temperature': None, 'date': '13-01-2022'}
{'avg_surface_temperature': None, 'date': '14-01-2022'}
{'avg_surface_temperature': None, 'date': '15-01-2022'}
{'avg_surface_temperature': None, 'date': '16-01-2022'}
{'avg_surface_temperature': None, 'date': '17-01-2022'}
{'avg_surface_temperature': None, 'date': '18-01

*i.	Find the top 10 records with the lowest GHI.*

In [15]:
# sort by GHI in ascending order 
# retrieve top 10 records only
result = collection.find().sort('GHI_w/m2', 1).limit(10)

for document in result:
    pprint(document)

{'GHI_w/m2': 47,
 '_id': ObjectId('6481357e25c02be8d1a09882'),
 'air_temperature_celcius': 5,
 'date': datetime.datetime(2022, 8, 2, 0, 0),
 'hotspots': [{'confidence': 94,
               'date': datetime.datetime(2022, 8, 2, 0, 0),
               'datetime': datetime.datetime(2022, 8, 2, 3, 45, 40),
               'latitude': -37.4796,
               'longitude': 141.9403,
               'surface_temperature_celcius': 87},
              {'confidence': 54,
               'date': datetime.datetime(2022, 8, 2, 0, 0),
               'datetime': datetime.datetime(2022, 8, 2, 3, 45),
               'latitude': -37.491,
               'longitude': 141.936,
               'surface_temperature_celcius': 40}],
 'max_wind_speed': 5.1,
 'precipitation': 0.0,
 'precipitation_flag': 'I',
 'relative_humidity': 38.6,
 'station': 948701,
 'windspeed_knots': 1.8}
{'GHI_w/m2': 48,
 '_id': ObjectId('6481357e25c02be8d1a09861'),
 'air_temperature_celcius': 5,
 'date': datetime.datetime(2022, 6, 30, 0, 0),


*j.	Find the records with a 24-hour precipitation recorded between 0.20 to 0.35.*

In [16]:
# 24 hour precipitation = D F G (flag)
result = collection.find({
    'precipitation_flag': {'$in': ['D', 'F', 'G']},
    'precipitation': {'$gte': 0.20, '$lte': 0.35}
})

for document in result:
    pprint(document)

{'GHI_w/m2': 157,
 '_id': ObjectId('6481357e25c02be8d1a097b9'),
 'air_temperature_celcius': 19,
 'date': datetime.datetime(2022, 1, 13, 0, 0),
 'hotspots': [],
 'max_wind_speed': 18.1,
 'precipitation': 0.31,
 'precipitation_flag': 'G',
 'relative_humidity': 54.1,
 'station': 948700,
 'windspeed_knots': 11.2}
{'GHI_w/m2': 146,
 '_id': ObjectId('6481357e25c02be8d1a09804'),
 'air_temperature_celcius': 17,
 'date': datetime.datetime(2022, 3, 29, 0, 0),
 'hotspots': [{'confidence': 69,
               'date': datetime.datetime(2022, 3, 29, 0, 0),
               'datetime': datetime.datetime(2022, 3, 29, 0, 48, 40),
               'latitude': -34.2648,
               'longitude': 141.6325,
               'surface_temperature_celcius': 51}],
 'max_wind_speed': 21.0,
 'precipitation': 0.24,
 'precipitation_flag': 'G',
 'relative_humidity': 49.9,
 'station': 948701,
 'windspeed_knots': 12.2}
{'GHI_w/m2': 166,
 '_id': ObjectId('6481357e25c02be8d1a0981a'),
 'air_temperature_celcius': 20,
 'date':

*3. The dataset we have is small for now. However, the number of stations and data points will increase significantly once we deploy our database and application in production. Considering those queries in Task 2 as typical use cases, think about how to optimise your mongodb database. Based on your data model and typical use cases, add simple or compound indexes to your collection. You also need to consider high data ingestion rate and design your indexes accordingly. Please justify your index design.*

#### `Justification:` ####

The 'date' field in the collection has been added as an index primarily because it serves as an identifier to establish the connection between climate data and hotspot data in the embedded data model where they are linked based on the shared date. Additionally, as the dataset grows and the rate of data ingestion increases, the 'date' index is able to handle the influx of new data well. Its presence enables expedited identification and retrieval of individual or groups of documents based on specific dates, facilitating real-time analysis and processing easily as each document in the collection has a unique date value.

Considering the typical queries used in Task 2, which often involve filtering or retrieving data based on specific dates or date ranges, the 'date' index greatly enhances query performance. By narrowing down the search space, the use of 'date' index minimizes the number of documents that need to be examined, resulting in faster query execution and improved overall efficiency. The following example (Query A in Task 2) is used to showcase the performance improvement achieved by adding the 'date' index to the collection.

#### Before adding 'date' index: ####
We can see that the number of documents being examined 'totalDocsExamined' is 366 which is the same as the total number of documents in the collection. This shows the inefficiency in querying the single document with the unique target date of 12-12-2022 as it searches over the entire collection which is time consuming.

In [17]:
# before creating date as index
collection.find({'date': target_date}, {'_id': 0, 'hotspots': 0}).explain()['executionStats']

{'executionSuccess': True,
 'nReturned': 1,
 'executionTimeMillis': 0,
 'totalKeysExamined': 0,
 'totalDocsExamined': 366,
 'executionStages': {'stage': 'PROJECTION_DEFAULT',
  'nReturned': 1,
  'executionTimeMillisEstimate': 0,
  'works': 368,
  'advanced': 1,
  'needTime': 366,
  'needYield': 0,
  'saveState': 0,
  'restoreState': 0,
  'isEOF': 1,
  'transformBy': {'_id': 0, 'hotspots': 0},
  'inputStage': {'stage': 'COLLSCAN',
   'filter': {'date': {'$eq': datetime.datetime(2022, 12, 12, 0, 0)}},
   'nReturned': 1,
   'executionTimeMillisEstimate': 0,
   'works': 368,
   'advanced': 1,
   'needTime': 366,
   'needYield': 0,
   'saveState': 0,
   'restoreState': 0,
   'isEOF': 1,
   'direction': 'forward',
   'docsExamined': 366}},
 'allPlansExecution': []}

#### After adding 'date' index: ####

We can see that the number of documents being examined 'totalDocsExamined' has been reduced to 1. This means that the 'date' index helps us to identify and retrieve the document with the specific date straightaway without the need to go over the entire collection, demonstrating the effectiveness of the 'date' index.

In [18]:
# create date as index
result = collection.create_index([('date', pymongo.ASCENDING)])
sorted(list(collection.index_information()))

['_id_', 'date_1']

In [19]:
# after creating date as index
collection.find({'date': target_date}, {'_id': 0, 'hotspots': 0}).explain()['executionStats']

{'executionSuccess': True,
 'nReturned': 1,
 'executionTimeMillis': 2,
 'totalKeysExamined': 1,
 'totalDocsExamined': 1,
 'executionStages': {'stage': 'PROJECTION_DEFAULT',
  'nReturned': 1,
  'executionTimeMillisEstimate': 3,
  'works': 2,
  'advanced': 1,
  'needTime': 0,
  'needYield': 0,
  'saveState': 0,
  'restoreState': 0,
  'isEOF': 1,
  'transformBy': {'_id': 0, 'hotspots': 0},
  'inputStage': {'stage': 'FETCH',
   'nReturned': 1,
   'executionTimeMillisEstimate': 3,
   'works': 2,
   'advanced': 1,
   'needTime': 0,
   'needYield': 0,
   'saveState': 0,
   'restoreState': 0,
   'isEOF': 1,
   'docsExamined': 1,
   'alreadyHasObj': 0,
   'inputStage': {'stage': 'IXSCAN',
    'nReturned': 1,
    'executionTimeMillisEstimate': 3,
    'works': 2,
    'advanced': 1,
    'needTime': 0,
    'needYield': 0,
    'saveState': 0,
    'restoreState': 0,
    'isEOF': 1,
    'keyPattern': {'date': 1},
    'indexName': 'date_1',
    'isMultiKey': False,
    'multiKeyPaths': {'date': []},
  